# Dataset for Pnoi experiments

> prepare master_pnoi_corpus.csv file that contains informant of entire pnoi-corpus

In [153]:
import pandas as pd
import numpy as np
import shutil
import glob
import json
import os

mkdir = lambda p: 0 if os.path.exists(p) else (os.makedirs(p), 1)[1]

## STATE VARIABLES

In [154]:
IS_DRY_RUN = False
IS_DUMMY_PATH = False

## PATHS

In [155]:
DATA_PATH = "DATA_DUMMY" if IS_DUMMY_PATH else "DATA_PNOISTOR"

REPORT_FOLDER = "./report"; mkdir(REPORT_FOLDER)

0

## Dataset Class

In [156]:
class DataStaticInfo:

    VER = "*"
    SEP = "-"
    META_SEP = "_"
    EXT_SEP = "."
    ANOT_LABELS = ["aa", "ee", "uu", "oo", "ii", "xx", "bb1", "bb2", "bb3", "bb4"]

    fkeys = {
        "APP_CODE": "app_code",
        "SID":"sub_id",
        "FCLASS": "file_class",
        "FID": "file_ID",
        "COMNT": "file_comment",
        "FFMT": "file_format",
        "FNAME": "file_name",
        "FPATH": "file_path",
        "FMATCH": "file_match"
    }

    

In [157]:
class DataExtractFiles(DataStaticInfo):

    ALL_FILES_DF: pd.DataFrame
    

    def __init__(self, path: str, ver="*") -> None:
        self.VER = ver

        all_files = glob.glob(f"{path}/pnoistor_{self.VER}/*/pnoistor_*")

        self.ALL_FILES_DF = self.make_files_df(all_files)


    def file_dict(self, fpath: str) -> dict:
        _, fname = os.path.split(fpath)

        match_key = fname.split(self.EXT_SEP)[0]

        fitems = fname.replace(self.EXT_SEP, self.SEP, 1).split(self.SEP) 
        fitems += [fname, fpath, match_key]
        fdict = dict(zip(self.fkeys.values(), fitems))
        return fdict
    
    def make_files_df(self, all_files: list) -> pd.DataFrame:
        files = [self.file_dict(fp) for fp in all_files]
        return pd.DataFrame(files)
    
    def get_file_extention(self) -> str:
        return self.ALL_FILES_DF[self.fkeys["FFMT"]].unique()
    
    def get_subject_list(self) -> list:
        return self.ALL_FILES_DF[self.fkeys["SID"]].unique()
    
pnoidata_files = DataExtractFiles(DATA_PATH, ver="*")


In [158]:
pnoidata_files.get_subject_list().__len__()

33

In [159]:
print(len(pnoidata_files.ALL_FILES_DF), "files")
pnoidata_files.ALL_FILES_DF.head()

564 files


,app_code,sub_id,file_class,file_ID,file_comment,file_format,file_name,file_path,file_match
0,pnoistor_jun2023,maitribrambhatt_540264cd,LBA_before_LL,b94e,comnt,wav,pnoistor_jun2023-maitribrambhatt_540264cd-LBA_...,DATA_PNOISTOR/pnoistor_jun2023/maitribrambhatt...,pnoistor_jun2023-maitribrambhatt_540264cd-LBA_...
1,pnoistor_jun2023,maitribrambhatt_540264cd,META,4122,comnt,meta.json,pnoistor_jun2023-maitribrambhatt_540264cd-META...,DATA_PNOISTOR/pnoistor_jun2023/maitribrambhatt...,pnoistor_jun2023-maitribrambhatt_540264cd-META...
2,pnoistor_jun2023,maitribrambhatt_540264cd,LBA_before_LU,f63f,comnt,wav,pnoistor_jun2023-maitribrambhatt_540264cd-LBA_...,DATA_PNOISTOR/pnoistor_jun2023/maitribrambhatt...,pnoistor_jun2023-maitribrambhatt_540264cd-LBA_...
3,pnoistor_jun2023,maitribrambhatt_540264cd,PFT_before,7ac0,comnt,pdf,pnoistor_jun2023-maitribrambhatt_540264cd-PFT_...,DATA_PNOISTOR/pnoistor_jun2023/maitribrambhatt...,pnoistor_jun2023-maitribrambhatt_540264cd-PFT_...
4,pnoistor_jun2023,maitribrambhatt_540264cd,META,4122,comnt,json,pnoistor_jun2023-maitribrambhatt_540264cd-META...,DATA_PNOISTOR/pnoistor_jun2023/maitribrambhatt...,pnoistor_jun2023-maitribrambhatt_540264cd-META...


In [160]:
pnoidata_files.get_file_extention()

array(['wav', 'meta.json', 'pdf', 'json', 'WAV', 'txt', 'tsv'],
      dtype=object)

In [161]:
class DataSourceFiles(DataStaticInfo):

    src_data_fname_regx = r"^pnoistor_(?!.*\.meta\.).+\.(wav|WAV|pdf|json|)$"

    ALL_FILES_DF: pd.DataFrame
    SOURCE_FILES_DF: pd.DataFrame

    def __init__(self, all_file_df: pd.DataFrame) -> None:
        self.ALL_FILES_DF = all_file_df

        self.SOURCE_FILES_DF = self.pair_src_anot_files()

    def make_source_files_df(self) -> pd.DataFrame:
        filt = self.ALL_FILES_DF.loc[:, self.fkeys["FNAME"]].str.match(self.src_data_fname_regx)

        df = self.ALL_FILES_DF[filt]
        return df
    
    def anot_path_join(self, p: pd.Series) -> str:
        fpath = os.path.join(os.path.split(p[self.fkeys['FPATH']])[0], p[f"anot--{self.fkeys['FNAME']}"])

        return fpath if os.path.exists(fpath) else False
    
    def pair_src_anot_files(self) -> pd.DataFrame:
        anot_format_mapping = { self.fkeys["FFMT"]: {'wav': 'txt', 'WAV': 'txt', 'pdf': 'tsv', "json": "meta.json"} }

        source_files_DF = self.make_source_files_df()
    
        anot_file_format_DF = source_files_DF.replace(anot_format_mapping) # anotation file extension

        # rename file_format to anot--file-format
        anot_fmt_column = f"anot--{self.fkeys['FFMT']}"
        anot_file_format_DF.rename(columns={self.fkeys["FFMT"]: anot_fmt_column}, inplace=True)
        anot_file_format_DF = anot_file_format_DF[[anot_fmt_column, self.fkeys["FMATCH"]]]
       
        
        # mergr source and anotation files
        src_data_files_DF = pd.merge(source_files_DF, anot_file_format_DF, on=self.fkeys["FMATCH"], how="left")
        anot_fname_column = f"anot--{self.fkeys['FNAME']}"

        src_data_files_DF[anot_fname_column] = src_data_files_DF[self.fkeys["FMATCH"]] + '.' + src_data_files_DF[anot_fmt_column]


        anot_fpath_column = f"anot--{self.fkeys['FPATH']}"
        anote_filepath_df = src_data_files_DF.apply(self.anot_path_join, axis=1).to_frame(name=anot_fpath_column)

        src_data_files_DF = pd.concat([src_data_files_DF, anote_filepath_df], axis=1)

        return src_data_files_DF

pnoidata_source_files = DataSourceFiles(pnoidata_files.ALL_FILES_DF)
pnoidata_source_files.SOURCE_FILES_DF

,app_code,sub_id,file_class,file_ID,file_comment,file_format,file_name,file_path,file_match,anot--file_format,anot--file_name,anot--file_path
0,pnoistor_jun2023,maitribrambhatt_540264cd,LBA_before_LL,b94e,comnt,wav,pnoistor_jun2023-maitribrambhatt_540264cd-LBA_...,DATA_PNOISTOR/pnoistor_jun2023/maitribrambhatt...,pnoistor_jun2023-maitribrambhatt_540264cd-LBA_...,txt,pnoistor_jun2023-maitribrambhatt_540264cd-LBA_...,False
1,pnoistor_jun2023,maitribrambhatt_540264cd,LBA_before_LU,f63f,comnt,wav,pnoistor_jun2023-maitribrambhatt_540264cd-LBA_...,DATA_PNOISTOR/pnoistor_jun2023/maitribrambhatt...,pnoistor_jun2023-maitribrambhatt_540264cd-LBA_...,txt,pnoistor_jun2023-maitribrambhatt_540264cd-LBA_...,False
2,pnoistor_jun2023,maitribrambhatt_540264cd,PFT_before,7ac0,comnt,pdf,pnoistor_jun2023-maitribrambhatt_540264cd-PFT_...,DATA_PNOISTOR/pnoistor_jun2023/maitribrambhatt...,pnoistor_jun2023-maitribrambhatt_540264cd-PFT_...,tsv,pnoistor_jun2023-maitribrambhatt_540264cd-PFT_...,False
3,pnoistor_jun2023,maitribrambhatt_540264cd,META,4122,comnt,json,pnoistor_jun2023-maitribrambhatt_540264cd-META...,DATA_PNOISTOR/pnoistor_jun2023/maitribrambhatt...,pnoistor_jun2023-maitribrambhatt_540264cd-META...,meta.json,pnoistor_jun2023-maitribrambhatt_540264cd-META...,DATA_PNOISTOR/pnoistor_jun2023/maitribrambhatt...
4,pnoistor_jun2023,maitribrambhatt_540264cd,LBA_before_RL,f315,comnt,wav,pnoistor_jun2023-maitribrambhatt_540264cd-LBA_...,DATA_PNOISTOR/pnoistor_jun2023/maitribrambhatt...,pnoistor_jun2023-maitribrambhatt_540264cd-LBA_...,txt,pnoistor_jun2023-maitribrambhatt_540264cd-LBA_...,False
...,...,...,...,...,...,...,...,...,...,...,...,...
300,pnoistor_may2023,valli_82ee8284,LBA_before_LL,4b19,comnt,wav,pnoistor_may2023-valli_82ee8284-LBA_before_LL-...,DATA_PNOISTOR/pnoistor_may2023/valli_82ee8284/...,pnoistor_may2023-valli_82ee8284-LBA_before_LL-...,txt,pnoistor_may2023-valli_82ee8284-LBA_before_LL-...,DATA_PNOISTOR/pnoistor_may2023/valli_82ee8284/...
301,pnoistor_may2023,valli_82ee8284,LBA_before_LU,6a5c,comnt,wav,pnoistor_may2023-valli_82ee8284-LBA_before_LU-...,DATA_PNOISTOR/pnoistor_may2023/valli_82ee8284/...,pnoistor_may2023-valli_82ee8284-LBA_before_LU-...,txt,pnoistor_may2023-valli_82ee8284-LBA_before_LU-...,DATA_PNOISTOR/pnoistor_may2023/valli_82ee8284/...
302,pnoistor_may2023,valli_82ee8284,PFT_before,4748,comnt,pdf,pnoistor_may2023-valli_82ee8284-PFT_before-474...,DATA_PNOISTOR/pnoistor_may2023/valli_82ee8284/...,pnoistor_may2023-valli_82ee8284-PFT_before-474...,tsv,pnoistor_may2023-valli_82ee8284-PFT_before-474...,DATA_PNOISTOR/pnoistor_may2023/valli_82ee8284/...
303,pnoistor_may2023,valli_82ee8284,META,18a6,comnt,json,pnoistor_may2023-valli_82ee8284-META-18a6-comn...,DATA_PNOISTOR/pnoistor_may2023/valli_82ee8284/...,pnoistor_may2023-valli_82ee8284-META-18a6-comnt,meta.json,pnoistor_may2023-valli_82ee8284-META-18a6-comn...,DATA_PNOISTOR/pnoistor_may2023/valli_82ee8284/...


In [162]:
import re

class ProcessPFTFiles(DataStaticInfo):

    SOURCE_FILES_DF: pd.DataFrame

    def __init__(self, all_src_file_df: pd.DataFrame) -> None:
        self.SOURCE_FILES_DF = all_src_file_df

        self.SOURCE_FILES_PFT_DF = self.make_pft_df()


    def read_pft_data(self, rv):
        file_path = rv[self.fkeys["FPATH"]]
        print

        # print(f"./{file_path}")
        anot_fpath = file_path.replace(".pdf", ".tsv")
        if not os.path.exists(anot_fpath):
            return
        
        # pft_files.append(anot_fpath)

        df = pd.read_csv(anot_fpath, sep="\t")
        file_info = {
            # f"file_path": file_path,
            self.fkeys["SID"]: rv[self.fkeys["SID"]],
        }
        pft_dict = {}
        for i, row in df.iterrows():
            for col in df.columns[1:]:
                val_typ = row["PFT"]
                if 'before' in rv[self.fkeys["FNAME"]]:
                    pft_key = f"{col}_{val_typ}_{'before'}"
                else:
                    pft_key = f"{col}_{val_typ}_{'after'}"

                pft_val = df.loc[i, col]

                pft_dict[pft_key] = pft_val

        pft_dict = pft_dict | file_info

        # df_reshape =  pd.DataFrame([pft_dict])
        return pft_dict
    
    def make_pft_df(self) -> pd.DataFrame:

        pft_tags = ["PFT_before", "PFT_after"]

        def pft_data_df(tag: str) -> pd.DataFrame:
            filt = self.SOURCE_FILES_DF[self.fkeys["FCLASS"]].str.contains(tag)
            src_PFT_data_DF = self.SOURCE_FILES_DF[filt]

            pft_dict_list = [self.read_pft_data(rv) for _, rv in src_PFT_data_DF.iterrows()]
            pft_dict_list = [x for x in pft_dict_list if x is not None]; pft_dict_list
            PFT_data_DF = pd.DataFrame(pft_dict_list)
            PFT_data_DF.fillna("-", inplace=True); PFT_data_DF.columns

            return PFT_data_DF
        
        src_data_files_DF = self.SOURCE_FILES_DF.copy()
        for tag in pft_tags:

            src_data_files_DF = src_data_files_DF.merge(pft_data_df(tag), how="left", on=[self.fkeys["SID"]])

        return src_data_files_DF
    

    def check_pft_typos(self, pft_files: list) -> None:
        pat = r'\b(?:FVE1|FEv1|raf)\b'
    
        for file in pft_files:
            with open(file, 'r') as fp:
                content = fp.read()

                match = re.findall(pat, content)
            
                if len(match) > 0:
                    print(match, f"./{file}")

        
    
pnoidata_pft_files = ProcessPFTFiles(pnoidata_source_files.SOURCE_FILES_DF)
pnoidata_pft_files.make_pft_df().columns.to_list()

['app_code',
 'sub_id',
 'file_class',
 'file_ID',
 'file_comment',
 'file_format',
 'file_name',
 'file_path',
 'file_match',
 'anot--file_format',
 'anot--file_name',
 'anot--file_path',
 'FEV1_val_before',
 'FVC_val_before',
 'ratio_val_before',
 'FEV1_ref_before',
 'FVC_ref_before',
 'ratio_ref_before',
 'FEV1_ref_after',
 'FVC_ref_after',
 'ratio_ref_after',
 'FEV1_val_after',
 'FVC_val_after',
 'ratio_val_after']

In [163]:
class ProcessMETAFiles(DataStaticInfo):

    SOURCE_FILES_DF: pd.DataFrame
    SOURCE_FILES_META_DF: pd.DataFrame
    
    def __init__(self, all_src_file_df: pd.DataFrame) -> None:
        self.SOURCE_FILES_DF = all_src_file_df
        
        self.SOURCE_FILES_META_DF = self.make_meta_df()

    def read_metadata(self, row):
        file_path = row[self.fkeys["FPATH"]]

        # file_info = { "file_path": file_path }

        with open(file_path, 'r') as f:
            # print(f"./{file_path}")
            data = json.load(f)
            bio_data: dict = data["subjectBiodata"] #| file_info
            survey_data_q = data["subjectSurvey"]["allQuestions"]
            survey_data_a = data["subjectSurvey"]["answeredQs"]

            
            bio_data_dict = [{ "qn": k, "Q": f"{k}?", "A": v } for k, v in sorted(bio_data.items())]
            survey_data_dict_q = [{ "qn": str(q["qno"]), "Q": q["question"] } for q in survey_data_q.values()]
            survey_data_dict = [{ "qn": str(q["qno"]), "Q": q["question"], "A": q["answer"] } for q in survey_data_a]
            survey_data_q_DF = pd.DataFrame(survey_data_dict_q)
            survey_data_a_DF = pd.DataFrame(bio_data_dict + survey_data_dict)

            survey_data_o_DF = survey_data_q_DF.merge(survey_data_a_DF, how="outer", on=["qn", "Q"])
            survey_data_o_DF.fillna("-", inplace=True)

            nfile_path = file_path.replace(".json", ".meta.json")
            # print(f"./{nfile_path}")
            survey_data_o_DF.set_index('qn').to_json(nfile_path, orient='index')

            return (bio_data, survey_data_o_DF)
        
    def make_meta_df(self) -> pd.DataFrame:
        filt = self.SOURCE_FILES_DF[self.fkeys["FCLASS"]] == "META"
        src_META_data_DF = self.SOURCE_FILES_DF[filt]

        all_META_data_DF = pd.DataFrame([self.read_metadata(row)[0] for _, row in src_META_data_DF.iterrows()])

        drop_cols = ["subjectSectionDone", "subjectRemunerationType", "subjectRemunerationDetails"]
        all_META_data_DF = all_META_data_DF.drop(columns=drop_cols)
        all_META_data_DF = all_META_data_DF.rename(columns={"firebaseId": self.fkeys["SID"]})

        src_data_files_DF = self.SOURCE_FILES_DF.copy()
        src_data_files_DF = src_data_files_DF.merge(all_META_data_DF, how="left", on=[self.fkeys["SID"]])

        return src_data_files_DF

pnoidata_meta_files = ProcessMETAFiles(pnoidata_pft_files.SOURCE_FILES_PFT_DF)
pnoidata_meta_files.make_meta_df().columns.to_list()

['app_code',
 'sub_id',
 'file_class',
 'file_ID',
 'file_comment',
 'file_format',
 'file_name',
 'file_path',
 'file_match',
 'anot--file_format',
 'anot--file_name',
 'anot--file_path',
 'FEV1_val_before',
 'FVC_val_before',
 'ratio_val_before',
 'FEV1_ref_before',
 'FVC_ref_before',
 'ratio_ref_before',
 'FEV1_ref_after',
 'FVC_ref_after',
 'ratio_ref_after',
 'FEV1_val_after',
 'FVC_val_after',
 'ratio_val_after',
 'subjectName',
 'subjectGender',
 'subjectAge',
 'subjectType',
 'subjectHeight',
 'subjectWeight']

In [168]:
from functools import reduce

class FinalProcessDataFiles(DataStaticInfo):

    merge_cols = [
    'app_code',
    'sub_id',

    'subjectName',
    'subjectGender',
    'subjectAge',
    'subjectType',
    'subjectHeight',
    'subjectWeight',

    'FEV1_val_before',
    'FVC_val_before',
    'ratio_val_before',

    'FEV1_ref_before',
    'FVC_ref_before',
    'ratio_ref_before',

    'FEV1_ref_after',
    'FVC_ref_after',
    'ratio_ref_after',

    'FEV1_val_after',
    'FVC_val_after',
    'ratio_val_after',
    ]


    drop_cols = ["file_format", "file_name", "file_class", "file_ID", "file_comment", "file_match"]


    SOURCE_FILES_DF: pd.DataFrame
    FCLASS_DF_LIST: list 
    MASTER_DATA_DF: pd.DataFrame
    MASTER_DATA_EXPORT_PATH: str

    def __init__(self, all_src_file_df: pd.DataFrame, export_path: str) -> None:
        self.SOURCE_FILES_DF = all_src_file_df
        self.MASTER_DATA_EXPORT_PATH = export_path

        self.MASTER_DATA_DF = self.merge_fclass_dfs()

    def seperate_fclass(self):

        fclass_DFs = []
        for fclass in self.SOURCE_FILES_DF[self.fkeys["FCLASS"]].unique():

            filt = self.SOURCE_FILES_DF[self.fkeys["FCLASS"]] == fclass
            fclass_df = self.SOURCE_FILES_DF[filt]

            # fclass_df = fclass_df.drop(columns=["index"])
            fclass_df.columns = fclass_df.columns.str.replace("file", f"{fclass}--file")

            fclass_DFs.append(fclass_df)

        return fclass_DFs
    
    col_order = [
        'index', 'app_code', 'sub_id',

        'subjectName', 'subjectGender', 'subjectAge',
        'subjectType', 'subjectHeight', 'subjectWeight',
        
        'META--file_path', 'anot--META--file_path',

        'FEV1_ref_before', 'FEV1_val_before',
        'FVC_ref_before', 'FVC_val_before',
        'ratio_ref_before', 'ratio_val_before',
        
        'FEV1_ref_after', 'FEV1_val_after',
        'FVC_ref_after', 'FVC_val_after',
        'ratio_ref_after', 'ratio_val_after',

        'VBA_before--file_path', 'anot--VBA_before--file_path',
        'VBA_after--file_path', 'anot--VBA_after--file_path',

        'LBA_before_LU--file_path', 'anot--LBA_before_LU--file_path',
        'LBA_before_RU--file_path', 'anot--LBA_before_RU--file_path',
        'LBA_before_LL--file_path', 'anot--LBA_before_LL--file_path',
        'LBA_before_RL--file_path', 'anot--LBA_before_RL--file_path',

        'LBA_after_LU--file_path', 'anot--LBA_after_LU--file_path',
        'LBA_after_RU--file_path', 'anot--LBA_after_RU--file_path',
        'LBA_after_LL--file_path', 'anot--LBA_after_LL--file_path',
        'LBA_after_RL--file_path', 'anot--LBA_after_RL--file_path',
        
        'PFT_before--file_path', 'anot--PFT_before--file_path',
        'PFT_after--file_path', 'anot--PFT_after--file_path',
        ]
            
    def merge_fclass_dfs(self):

        fclass_DFs = self.seperate_fclass()

        df_merged = reduce(lambda  left, right: pd.merge(left, right, on=self.merge_cols, how='outer'), fclass_DFs)

        df_cols = list(df_merged.columns.delete(df_merged.columns.str.contains("|".join(self.drop_cols))))

        df_merged = df_merged[df_cols]

        df_merged = df_merged.fillna(False)
        df_merged = df_merged.replace(False, "-")

        df_merged.sort_values(["subjectType", "subjectGender", "subjectAge"], inplace=True)

        df_merged.reset_index(inplace=True)
        # df_merged.drop(columns=["index"], inplace=True)
        # df_merged.index.name = "index"

        df_merged = df_merged[self.col_order]

        print(list(df_merged.columns))
        df_merged.to_csv(self.MASTER_DATA_EXPORT_PATH, index=True)

        return df_merged

export_path = f"{REPORT_FOLDER}/pnoicorpus_muster.csv"
pnoidata_final_process = FinalProcessDataFiles(pnoidata_meta_files.SOURCE_FILES_META_DF, export_path)
# pnoidata_final_process.seperate_fclass()[0].columns.to_list()

# pnoidata_final_process.SOURCE_FILES_DF

pnoidata_final_process.MASTER_DATA_DF

['index', 'app_code', 'sub_id', 'subjectName', 'subjectGender', 'subjectAge', 'subjectType', 'subjectHeight', 'subjectWeight', 'META--file_path', 'anot--META--file_path', 'FEV1_ref_before', 'FEV1_val_before', 'FVC_ref_before', 'FVC_val_before', 'ratio_ref_before', 'ratio_val_before', 'FEV1_ref_after', 'FEV1_val_after', 'FVC_ref_after', 'FVC_val_after', 'ratio_ref_after', 'ratio_val_after', 'VBA_before--file_path', 'anot--VBA_before--file_path', 'VBA_after--file_path', 'anot--VBA_after--file_path', 'LBA_before_LU--file_path', 'anot--LBA_before_LU--file_path', 'LBA_before_RU--file_path', 'anot--LBA_before_RU--file_path', 'LBA_before_LL--file_path', 'anot--LBA_before_LL--file_path', 'LBA_before_RL--file_path', 'anot--LBA_before_RL--file_path', 'LBA_after_LU--file_path', 'anot--LBA_after_LU--file_path', 'LBA_after_RU--file_path', 'anot--LBA_after_RU--file_path', 'LBA_after_LL--file_path', 'anot--LBA_after_LL--file_path', 'LBA_after_RL--file_path', 'anot--LBA_after_RL--file_path', 'PFT_befo

,index,app_code,sub_id,subjectName,subjectGender,subjectAge,subjectType,subjectHeight,subjectWeight,META--file_path,...,LBA_after_RU--file_path,anot--LBA_after_RU--file_path,LBA_after_LL--file_path,anot--LBA_after_LL--file_path,LBA_after_RL--file_path,anot--LBA_after_RL--file_path,PFT_before--file_path,anot--PFT_before--file_path,PFT_after--file_path,anot--PFT_after--file_path
0,15,pnoistor_feb2023,shreyamgupta_78aa423a,Shreyam Gupta,Female,19,Control,158,70,DATA_PNOISTOR/pnoistor_feb2023/shreyamgupta_78...,...,-,-,-,-,-,-,DATA_PNOISTOR/pnoistor_feb2023/shreyamgupta_78...,DATA_PNOISTOR/pnoistor_feb2023/shreyamgupta_78...,-,-
1,1,pnoistor_jun2023,aditis_96917e0d,Aditi S,Female,20,Control,164,64,DATA_PNOISTOR/pnoistor_jun2023/aditis_96917e0d...,...,-,-,-,-,-,-,DATA_PNOISTOR/pnoistor_jun2023/aditis_96917e0d...,-,-,-
2,7,pnoistor_dec01,sannashoukat_5213fe84,Sanna Shoukat,Female,21,Control,166,55,DATA_PNOISTOR/pnoistor_dec01/sannashoukat_5213...,...,-,-,-,-,-,-,DATA_PNOISTOR/pnoistor_dec01/sannashoukat_5213...,DATA_PNOISTOR/pnoistor_dec01/sannashoukat_5213...,-,-
3,14,pnoistor_feb2023,sananaushad_80e84b51,Sana Naushad,Female,21,Control,158,60,DATA_PNOISTOR/pnoistor_feb2023/sananaushad_80e...,...,-,-,-,-,-,-,DATA_PNOISTOR/pnoistor_feb2023/sananaushad_80e...,DATA_PNOISTOR/pnoistor_feb2023/sananaushad_80e...,-,-
4,5,pnoistor_dec01,saikeerthanaarun_3364bc1a,Sai Keerthana Arun,Female,22,Control,172,58,DATA_PNOISTOR/pnoistor_dec01/saikeerthanaarun_...,...,-,-,-,-,-,-,-,-,-,-
5,20,pnoistor_oct07,sharmisthachakrabarti_80531fda,Sharmistha Chakrabarti,Female,42,Control,150,57,DATA_PNOISTOR/pnoistor_oct07/sharmisthachakrab...,...,-,-,-,-,-,-,DATA_PNOISTOR/pnoistor_oct07/sharmisthachakrab...,-,-,-
6,6,pnoistor_dec01,kumarchowdam_53f32e31,Kumar Chowdam,Male,21,Control,162,60,DATA_PNOISTOR/pnoistor_dec01/kumarchowdam_53f3...,...,-,-,-,-,-,-,DATA_PNOISTOR/pnoistor_dec01/kumarchowdam_53f3...,DATA_PNOISTOR/pnoistor_dec01/kumarchowdam_53f3...,-,-
7,17,pnoistor_feb2023,darshanvshettar_843c416b,Darshan V Shettar,Male,21,Control,178,64,DATA_PNOISTOR/pnoistor_feb2023/darshanvshettar...,...,-,-,-,-,-,-,DATA_PNOISTOR/pnoistor_feb2023/darshanvshettar...,DATA_PNOISTOR/pnoistor_feb2023/darshanvshettar...,-,-
8,18,pnoistor_feb2023,atifahmed_ebb5032c,Atif ahmed,Male,21,Control,170,70,DATA_PNOISTOR/pnoistor_feb2023/atifahmed_ebb50...,...,-,-,-,-,-,-,DATA_PNOISTOR/pnoistor_feb2023/atifahmed_ebb50...,DATA_PNOISTOR/pnoistor_feb2023/atifahmed_ebb50...,-,-
9,26,pnoistor_oct07,johnkiranborugada_e390538c,John Kiran Borugada,Male,21,Control,172,70,DATA_PNOISTOR/pnoistor_oct07/johnkiranborugada...,...,-,-,-,-,-,-,DATA_PNOISTOR/pnoistor_oct07/johnkiranborugada...,DATA_PNOISTOR/pnoistor_oct07/johnkiranborugada...,-,-
